<a href="https://colab.research.google.com/github/vladgap/LLC/blob/main/Rotem_LLC_Washing_Simulation_OOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn import preprocessing
import matplotlib.pyplot as plt

from scipy import optimize

In [2]:
!pip install MLNN
from MLNN import*

In [38]:
!wget -O LLC.py "https://raw.githubusercontent.com/vladgap/LLC/main/LLC_versions/LLC_5_1.py"
import LLC

--2025-03-05 15:42:49--  https://raw.githubusercontent.com/vladgap/LLC/main/LLC_versions/LLC_5_1.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22228 (22K) [text/plain]
Saving to: ‘LLC.py’

LLC.py              100%[===================>]  21.71K  --.-KB/s    in 0.002s  

2025-03-05 15:42:49 (12.9 MB/s) - ‘LLC.py’ saved [22228/22228]



In [4]:
def CopyPasteToArray(a):
  if a.startswith('\n'):
    b=a[1:]
  else:
    b=a
  if b.endswith('\n'):
    b=b[:-1]
  c=b.replace('\t',',')
  d=c.split('\n')
  f=[]
  for e in d:
    if e.replace(',','').replace('.','').isdigit(): # only digits no letters
      f.append(list(eval(e)))
    else:
      f.append(e.split(','))
  return f

## EQUIL_rotem_washing
ניסויי הפרדה עם ממס מטוהר

In [5]:
Xscale=[21.97576471,  0.05575882], [15.2853469,   0.05047379]
Tscale=[1.89941176, 11.19176471,  1.4,         0.02921412], [2.40216868, 1.27085536, 0.17875386, 0.01997862]

scaler_X3 = preprocessing.StandardScaler().fit([[0,0]])
scaler_X3_2 = preprocessing.StandardScaler().fit([[0,0]]) # מים בקלה חושבו רק על נתונים "טובים
scaler_T3_1 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_2 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_3 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_4 = preprocessing.StandardScaler().fit([[0]])

scaler_X3.mean_, scaler_X3.scale_ = Xscale[0], Xscale[1]
scaler_X3_2.mean_, scaler_X3_2.scale_ = [27.03608333,  0.0474    ], [14.68499343,  0.04213688] # מים בקלה חושבו רק על נתונים "טובים
scaler_T3_1.mean_, scaler_T3_1.scale_=[Tscale[0][0]], [Tscale[1][0]]
# scaler_T3_2.mean_, scaler_T3_2.scale_=[Tscale[0][1]], [Tscale[1][1]]
scaler_T3_2.mean_, scaler_T3_2.scale_=[10.81], [1.3091664] # מים בקלה חושבו רק על נתונים "טובים
scaler_T3_3.mean_, scaler_T3_3.scale_=[Tscale[0][2]], [Tscale[1][2]]
scaler_T3_4.mean_, scaler_T3_4.scale_=[Tscale[0][3]], [Tscale[1][3]]

In [6]:
# @title class EQUIL_rotem_washing1
class EQUIL_rotem_washing1: # ניסויי הפרדה עם ממס מטוהר
# מודל מקורי
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp3_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_1=[[[-0.2328053114817587, 1.4197258515721494],
  [-0.013739923476318716, -0.009509632921841494]],
 [[1.5848282259786703, -0.9625030403646567]],
 [[-1.3007118041663503], [1.5725920350068667]],
 [[1.717]]]
        bp3_1.import_weights(manual_weights3_1)
        predics3_1=scaler_T3_1.inverse_transform(bp3_1.run(scaler_X3.transform(x[:,[1,2]])))

        # water_org
        bp3_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_2=[[[0.3909002105371126, -0.8406267181665068],
  [-0.4530958444599508, 0.704664998208326]],
 [[-0.09671769242746425, 0.7927475123043677]],
 [[0.48441213405704675], [-0.7676170242577324]],
 [[0.6435152247205708]]]
        bp3_2.import_weights(manual_weights3_2)
        predics3_2=scaler_T3_2.inverse_transform(bp3_2.run(scaler_X3_2.transform(x[:,[1,2]]))) # מים בקלה חושבו רק על נתונים "טובים

        # solvent_aq
        bp3_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_3=[[[0.6217022345240342, 2.8386999598609814],
  [-0.6968938452576019, 5.596033242601742]],
 [[2.7109590937198265, 0.22193377952697202]],
 [[-0.9372090292368469], [-0.33293100862641933]],
 [[3.277263990359097]]]
        bp3_3.import_weights(manual_weights3_3)
        predics3_3=scaler_T3_3.inverse_transform(bp3_3.run(scaler_X3.transform(x[:,[1,2]])))

        # SO4_org
        bp3_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_4=[[[-0.10168260750729177, 1.5507975795954547],
  [-0.02429351337425621, -5.794943503077378e-05]],
 [[1.5556257229985646, -0.978368082003519]],
 [[-1.3219817921544152], [1.6394490601340548]],
 [[1.5742747334295601]]]
        bp3_4.import_weights(manual_weights3_4)
        predics3_4=scaler_T3_4.inverse_transform(bp3_4.run(scaler_X3.transform(x[:,[1,2]])))

        self.xout = np.array([predics3_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics3_2[:,0], predics3_1[:,0], predics3_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

In [7]:
# @title class EQUIL_rotem_washing2
class EQUIL_rotem_washing2: # ניסויי הפרדה עם ממס מטוהר
# מודל עם מים בקלה 12% גורף
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp3_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_1=[[[-0.2328053114817587, 1.4197258515721494],
  [-0.013739923476318716, -0.009509632921841494]],
 [[1.5848282259786703, -0.9625030403646567]],
 [[-1.3007118041663503], [1.5725920350068667]],
 [[1.717]]]
        bp3_1.import_weights(manual_weights3_1)
        predics3_1=abs(scaler_T3_1.inverse_transform(bp3_1.run(scaler_X3.transform(x[:,[1,2]]))))

        # water_org
        bp3_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_2=[[[0.3909002105371126, -0.8406267181665068],
  [-0.4530958444599508, 0.704664998208326]],
 [[-0.09671769242746425, 0.7927475123043677]],
 [[0.48441213405704675], [-0.7676170242577324]],
 [[0.6435152247205708]]]
        bp3_2.import_weights(manual_weights3_2)
        # predics3_2=abs(scaler_T3_2.inverse_transform(bp3_2.run(scaler_X3_2.transform(x[:,[1,2]])))) # מים בקלה חושבו רק על נתונים "טובים
        predics3_2=np.ones([len(x),1])*12 #  תוצאות אנליטיות לא מדויקות. לקחתי 12 באופן גורף

        # solvent_aq
        bp3_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_3=[[[0.6217022345240342, 2.8386999598609814],
  [-0.6968938452576019, 5.596033242601742]],
 [[2.7109590937198265, 0.22193377952697202]],
 [[-0.9372090292368469], [-0.33293100862641933]],
 [[3.277263990359097]]]
        bp3_3.import_weights(manual_weights3_3)
        predics3_3=abs(scaler_T3_3.inverse_transform(bp3_3.run(scaler_X3.transform(x[:,[1,2]]))))

        # SO4_org
        bp3_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_4=[[[-0.10168260750729177, 1.5507975795954547],
  [-0.02429351337425621, -5.794943503077378e-05]],
 [[1.5556257229985646, -0.978368082003519]],
 [[-1.3219817921544152], [1.6394490601340548]],
 [[1.5742747334295601]]]
        bp3_4.import_weights(manual_weights3_4)
        predics3_4=abs(scaler_T3_4.inverse_transform(bp3_4.run(scaler_X3.transform(x[:,[1,2]]))))

        self.xout = np.array([predics3_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics3_2[:,0], predics3_1[:,0], predics3_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

In [8]:
# @title class EQUIL_rotem_washing3
class EQUIL_rotem_washing3: # ניסויי הפרדה עם ממס מטוהר
# מודל עם מים בקלה 12% גורף
# שינוי במשקלים של ממס בכבדה. שיהיה יותר אופקי
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp3_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_1=[[[-0.2328053114817587, 1.4197258515721494],
  [-0.013739923476318716, -0.009509632921841494]],
 [[1.5848282259786703, -0.9625030403646567]],
 [[-1.3007118041663503], [1.5725920350068667]],
 [[1.717]]]
        bp3_1.import_weights(manual_weights3_1)
        predics3_1=abs(scaler_T3_1.inverse_transform(bp3_1.run(scaler_X3.transform(x[:,[1,2]]))))

        # water_org
        bp3_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_2=[[[0.3909002105371126, -0.8406267181665068],
  [-0.4530958444599508, 0.704664998208326]],
 [[-0.09671769242746425, 0.7927475123043677]],
 [[0.48441213405704675], [-0.7676170242577324]],
 [[0.6435152247205708]]]
        bp3_2.import_weights(manual_weights3_2)
        # predics3_2=abs(scaler_T3_2.inverse_transform(bp3_2.run(scaler_X3_2.transform(x[:,[1,2]])))) # מים בקלה חושבו רק על נתונים "טובים
        predics3_2=np.ones([len(x),1])*12 #  תוצאות אנליטיות לא מדויקות. לקחתי 12 באופן גורף

        # solvent_aq
        bp3_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu') # modified solvent_aq weights
        manual_weights3_3=[[[-7.637254337641781, -0.490322880885377],
  [-1.8058063735873857, -0.10159866035465558]],
 [[-0.843126492292387, -2.3299686091075773]],
 [[0.2643116291427324], [0.4479958576488306]],
 [[-0.7056420231884761]]]
        bp3_3.import_weights(manual_weights3_3)
        predics3_3=abs(scaler_T3_3.inverse_transform(bp3_3.run(scaler_X3.transform(x[:,[1,2]]))))

        # SO4_org
        bp3_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_4=[[[-0.10168260750729177, 1.5507975795954547],
  [-0.02429351337425621, -5.794943503077378e-05]],
 [[1.5556257229985646, -0.978368082003519]],
 [[-1.3219817921544152], [1.6394490601340548]],
 [[1.5742747334295601]]]
        bp3_4.import_weights(manual_weights3_4)
        predics3_4=abs(scaler_T3_4.inverse_transform(bp3_4.run(scaler_X3.transform(x[:,[1,2]]))))

        self.xout = np.array([predics3_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics3_2[:,0], predics3_1[:,0], predics3_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

# Battery simulation

In [9]:
# @title class Optimal_Washing_Battery
class Optimal_Washing_Battery:
    '''
    A class representing an optimal washing battery for a chemical plant process.
    It calculates and optimizes various parameters related to the washing process.
    '''
    def __init__(self, plant_data, stages_num=6):
        '''
        Initialize the Optimal_Washing_Battery with plant data.
        Performs initial calculations, sets up the plant input data, and runs optimizations.

        Parameters:
        plant_data (list): Contains PS_P2O5, WS_P2O5, W_flow, Prod_flow, Prod_P2O5 values
        stages_num (int): Number of stages in the battery (default is 6)
        '''

        self.stages_num=stages_num
        self.plant_data = plant_data
        PS_P2O5, PS_SO4, WS_P2O5, W_flow, W_SO4, Prod_flow, Prod_P2O5 = self.plant_data
        slope_process=(PS_P2O5-WS_P2O5)/Prod_P2O5
        PS_flow=(Prod_flow*Prod_P2O5+WS_P2O5*(W_flow-Prod_flow))/(PS_P2O5-WS_P2O5)
        WS_flow=PS_flow+W_flow-Prod_flow
        P2O5_recovery=Prod_flow*Prod_P2O5/(PS_flow*PS_P2O5)
        self.plant_input_data = PS_flow, W_flow, PS_P2O5, PS_SO4, W_SO4
        # print ('Purified solvent flow =',f"{PS_flow:.1f}",'ton/hr,  ', 'Washed solvent flow =',f"{WS_flow:.1f}",'ton/hr  ', 'P2O5 recovery =',f"{P2O5_recovery:.1%}")

        plant_data_table=pd.DataFrame([PS_flow,PS_P2O5,WS_flow,WS_P2O5,W_flow,Prod_flow,Prod_P2O5,f"{P2O5_recovery:.1%}",6,'---', '---'],
                                index=['PS_flow','PS_P2O5','WS_flow','WS_P2O5','W_flow','Prod_flow','Prod_P2O5','P2O5_recovery','Stages_num','Efficiency','Equilibrium'],
                                columns=['Plant data'])

        print ('Calculating entrainment = f(slope,eff) parameters')
        C=self.__calculate_C()

        print ('Calculating optimal eff and entrain')
        optimal_params=self.__calculate_optimal_params(C, slope_process)
        opt_eff, opt_entrain = optimal_params

        print ('Calculating entrainment optimal battery simulation')
        optimal_bat=LLC.Battery(stages_num=self.stages_num, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,PS_SO4], xin=[0,0,W_SO4], EQUIL=EQUIL_rotem_washing3, eff=opt_eff, convergence=1E-3, entrainment_perc_out=[opt_entrain,0])
        self.bat=optimal_bat
        print ('Ready')

        self.summary_table=self.__prepare_summary_table(optimal_bat, plant_data_table)
        # self.summary_table

        self.profile_table=LLC.BatteryTable(optimal_bat, show=False).data
        self.full_profile_table=LLC.BatteryTableFull(optimal_bat, show=False).data

    def __calculate_C(self,entr_initial=3.6, eff_init=0.77):
        '''
        Calculate parameters for the relationship between slope, entrainment, and efficiency.
        This method runs simulations with different entrainment and efficiency values to find
        coefficients for the equation: slope = a + b * entr + c * eff + d * eff^2

        Parameters:
        entr_initial (float): Initial entrainment value
        eff_init (float): Initial efficiency value

        Returns:
        numpy.ndarray: Coefficients [a, b, c, d] for the slope equation

        נדרש להתאים את המודל לנתוני התהליך. סחיפות ויעילות הם נתוני כניסה למודל, חייבים למצוא אותם ע"י נצבה והרצה. הדרך ישרה היא הרצת אופטימיזציה עם שני משתנים וחישוב שגיאה יחסית לנתוני תהליך. ההתכנסות איטית ולא מדוייקת.

        נמצא לפני זה שהתלות של השיפוע של קו הפעולה תלוי באחוז סחיפות בקשר ליניארי וביעילות בקשר
        מרובע. מחשבים פרמטרים של פונקציה זאת

        slope = a + b * entr + c * eff + d * eff^2 ==> entr = (slope - a - c * eff - d * eff^2)/ b

        בשלב ראשון מוסיפים עמודה עם יעילות בריבוע

        מתוך הריכוזים של התהליך מחשבים שיפוע של התהליך. מוצאים קשר בין סחיפות ויעילות.

        entrain=(slope_proc - C[0] - C[2] * eff - C[3] * eff**2)/C[1]

        האופטימיזציה הופכת לאחת עם משתנה בודד.
        '''

        PS_flow, W_flow, PS_P2O5, PS_SO4, W_SO4 = self.plant_input_data
        slopes=[]
        # entr_initial, eff_init = 3.6, 0.77
        for entr in [entr_initial, entr_initial*.75]: # לינארי -- מספיק 2
          for eff in [eff_init,eff_init*.85,eff_init*1.1]: # פורבולה -- צריך 3
            bat=LLC.Battery(stages_num=self.stages_num, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,PS_SO4], xin=[0,0,W_SO4], EQUIL=EQUIL_rotem_washing3, eff=eff, convergence=1E-3, entrainment_perc_out=[entr,0])
            slope=(bat.yin_list[0][1]-bat.yout_list[-1][1])/(bat.xout_list[0][1]-bat.xin_list[-1][1])
            slopes.append([entr, eff, slope])
            # print ('entr:', entr, '  eff:', eff, '  slope:', slope)
            print ('.', end=' ')
        print ('\n')
        arr=np.array(slopes)
        arr_mod=np.c_[np.ones([len(arr),1]),arr[:,0:2], arr[:,1]**2, arr[:,2]] # entrainment approximated to porabola
        rows=[0,1,2,3] # 4 rows for 4 unknowns
        A=arr_mod[rows,0:4]
        B=arr_mod[rows,4]
        C=np.dot(np.linalg.inv(A),B) # פתרון מערכת משוואות.  מחשבים ערכים של עמודה אחרונה (קו הפעולה) מתוך עמודות ראשונות
        return C

    def __calculate_optimal_params(self, C, slope_process):
        '''
        Calculate optimal efficiency and entrainment parameters.
        Uses optimization to find the best efficiency value, then calculates the corresponding entrainment.

        Parameters:
        C (numpy.ndarray): Coefficients from __calculate_C method
        slope_process (float): Slope of the process line

        Returns:
        list: Optimal efficiency and entrainment values
        '''

        bounds=[(0.5,1)]
        sol=optimize.direct(self.__P2O5_calc, bounds=bounds, args=(C, slope_process), f_min=0, f_min_rtol=0.001, maxiter=100, locally_biased=False)
        eff=sol.x[0]
        entrain=(slope_process-C[0]-C[2]*eff-C[3]*eff**2)/C[1]
        print ('\n', 'optimal data:\n', 'eff:', sol.x[0], 'entrain:', entrain, 'convergence:', sol.fun)
        return [eff, entrain]

    def __P2O5_calc(self, x, C, slope_process):
        '''
        Objective function for optimization.
        Calculates the error between the model's P2O5 output and the actual plant data.

        Parameters:
        x (list): Contains the efficiency value
        C (numpy.ndarray): Coefficients from __calculate_C method
        slope_process (float): Slope of the process line

        Returns:
        float: Absolute error between model and plant P2O5 values
        '''

        PS_flow, W_flow, PS_P2O5, PS_SO4, W_SO4 = self.plant_input_data
        eff = x[0]
        entrain=(slope_process-C[0]-C[2]*eff-C[3]*eff**2)/C[1]
        bat=LLC.Battery(stages_num=self.stages_num, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,PS_SO4], xin=[0,0,W_SO4], EQUIL=EQUIL_rotem_washing3, eff=eff, convergence=1E-3, entrainment_perc_out=[entrain,0])
        error=abs((Prod_P2O5-bat.xout_list[0][1]/98*71)/Prod_P2O5)
        # print (eff, entrain, bat.yout_list[-1][1]/98*71, bat.xout_list[0][1]/98*71, error, bat.runs, ',')
        print ('.', end=' ')
        return error

    def __prepare_summary_table(self, bat, plant_data_table):
        '''
        Prepare a summary table comparing plant data with model data.

        Parameters:
        bat (LLC.Battery): Optimized battery object
        plant_data_table (pandas.DataFrame): Table containing plant data

        Returns:
        pandas.DataFrame: Summary table with plant and model data
        '''

        summary_table = plant_data_table.copy()
        summary_table['Model data']=[bat.Oin_list[0],
                                        bat.yin_list[0][1]/98*71,
                                        bat.Oout_list[-1],bat.yout_list[-1][1]/98*71,
                                        bat.Ain_list[-1],
                                        bat.Aout_list[0],
                                        bat.xout_list[0][1]/98*71,
                                        f"{bat.Aout_list[0]*bat.xout_list[0][1]/(bat.Oin_list[0]*bat.yin_list[0][1]):.1%}",
                                        f"{bat.stages_num:.0f}",
                                        bat.eff_list[0,1],
                                        str(bat.EQUIL)[17:-2]]
        return summary_table

# Washing 6 stages

In [10]:
PS_P2O5=11.6  # %P2O5 in purified silvent
PS_SO4=0.07  # %SO4 in purified solvent
WS_P2O5=0.2  # %P2O5 in washed solvent
W_flow=23.4  # Water flow
W_SO4=0.02  # %SO4 in water
Prod_flow=48.4 # =36.7*1.32 Product acid flow
Prod_P2O5=37.6  # %P2O5 in product acid
plant_data = [PS_P2O5, PS_SO4, WS_P2O5, W_flow, W_SO4, Prod_flow, Prod_P2O5]

In [11]:
opt_bat=Optimal_Washing_Battery(plant_data, stages_num=6)

Calculating entrainment = f(slope,eff) parameters
. . . . . . 

Calculating optimal eff and entrain
100 iterations!
. . . . . . . . . 
 optimal data:
 eff: 0.7685185185185185 entrain: 3.6340564091806447 convergence: 0.0004520221213951232
Calculating entrainment optimal battery simulation
Ready


In [12]:
opt_bat.summary_table

,Plant data,Model data
PS_flow,159.196491,159.196491
PS_P2O5,11.6,11.6
WS_flow,134.196491,129.754408
WS_P2O5,0.2,0.205547
W_flow,23.4,23.4
Prod_flow,48.4,48.122576
Prod_P2O5,37.6,37.583004
P2O5_recovery,98.5%,97.9%
Stages_num,6,6
Efficiency,---,0.768519


In [13]:
opt_bat.profile_table

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5,Stage 6
"Org in, ton/hr",159.196491,148.500785,141.283446,134.467694,132.008448,130.859187
"y1 in, %",12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
"y2 in, %",16.011268,10.905659,7.056373,3.053326,1.464370,0.694615
"y3 in, %",0.070000,0.088341,0.069994,0.038257,0.024417,0.019125
"Aq in, ton/hr",42.824965,35.341326,28.286670,25.731393,24.545350,23.400000
"x1 in, %",1.272657,1.272957,1.273537,1.499059,1.700473,0.000000
"x2 in, %",43.128961,32.974460,18.367799,8.891569,3.295717,0.000000
"x3 in, %",0.286259,0.267624,0.161201,0.081603,0.040609,0.020000
"Org out, ton/hr",148.502225,141.281387,134.472351,132.005561,130.862295,129.754408
"y1 out, %",12.000000,12.000000,12.000000,12.000000,12.000000,12.000000


In [14]:
opt_bat.profile_table.to_csv('Profile_table.csv')

## Operation Curve

In [15]:
# @markdown Profile data
data=CopyPasteToArray('''
x	y
0	0.0531
3.01	0.26
10.8	1.22
20.69	6.02
33.12	7.77
34.9	8.86
35.81	10.3
''')
profile_data1=pd.DataFrame(data[1:], columns=data[0])
profile_data1.to_csv('Profile_data.csv')

In [16]:
## P2O5
bat=opt_bat.bat
summary_table=opt_bat.summary_table
fig10_1 = go.Figure()
n=1 #H3PO4
fig10_1.add_trace(go.Scatter(x=[0,summary_table['Plant data'].loc['Prod_P2O5']], y=[summary_table['Plant data'].loc['WS_P2O5'],summary_table['Plant data'].loc['PS_P2O5']], name='Plant data', marker_color='orange'))
fig10_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))
fig10_1.add_trace(go.Scatter(x=profile_data1.x, y=profile_data1.y, name='Plant profile data', marker_color='grey', visible='legendonly'))

fig10_1.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig10_1.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
# fig10_1.update_layout(title='', width=1200, height=600,)
fig10_1.show()

## Equilibrium data

In [17]:
data=CopyPasteToArray('''
H3PO4_aq	SO4_aq	H3PO4_org	water_org	solvent_aq	SO4_org	water_org-original
0.011	0.0932	0.072	8.69	1.70	0.0186	12.09
10.325	0.0017	0.229	10.66	1.52	0.02	10.66
10.380	0.0990	0.269	8.31	1.50	0.0208	10.41
19.821	0.0031	0.635	11.3	1.34	0.01884	11.3
19.710	0.0928	0.773	10.36	1.32	0.0195	12.34
28.710	0.0057	1.435	11.96	1.28	0.01978	11.96
27.882	0.0858	1.325	9.7	1.34	0.0222	11.1
35.611	0.0071	2.871	11.94	1.28	0.036	11.94
35.887	0.0846	2.981	12.02	1.24	0.0374	12.6
43.065	0.0082	5.328	12.12	1.28	0.057	12.12
43.065	0.0802	5.604	10.38	1.06	0.0608	11.7
49.966	0.0074	8.723	12.28	1.28	0.0872	12.28
0.017	0.1540	0.042	11.48	1.46	0.0164	13.28
4.831	0.0020	0.146	12.7	1.78	0.01302	12.6
4.941	0.1078	0.173	12.06	1.66	0.0139	11.84
19.600	0.0033	0.828	11.84	1.38	0.01688	11.42
19.766	0.1120	0.856	12.46	1.38	0.01832	11.2
''')
# data
# data2=pd.DataFrame(data, columns=['H3PO4_aq',	'SO4_aq',	'H3PO4_org',	'water_org',	'solvent_aq',	'SO4_org'])
data3=pd.DataFrame(data[1:], columns=data[0])
# data3_names=CopyPasteToArray('''
# wp12	wp21	wp22	wp31	wp32	wp41	wp42	wp51	wp52	wp61	wp62	wp71	wp13	ww11	ww12	ww21	ww22
# ''')

In [18]:
data3.to_csv('Lab_data.csv')

In [19]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh4 = x_mesh.flatten().reshape(31*31,1)
y_mesh4 = y_mesh.flatten().reshape(31*31,1)
mesh4=np.concatenate((x_mesh4, y_mesh4), axis=1)

In [20]:
a=np.concatenate((np.zeros([len(mesh4),1]),mesh4), axis=1)

In [21]:
mesh_predict_x=np.array([EQUIL_rotem_washing3([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3([row]).yout for row in a])
mesh_predics4_1=mesh_predict_y[:,:,1]
mesh_predics4_2=mesh_predict_y[:,:,0]
mesh_predics4_3=mesh_predict_x[:,:,0]
mesh_predics4_4=mesh_predict_y[:,:,2]

In [22]:
fig_equil10_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=opt_bat.bat
# H3PO4_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil10_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil10_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil10_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil10_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil10_1.update_scenes(camera_projection_type="orthographic")

fig_equil10_1.show()

# Washing 1+5 stages

In [23]:
PS_P2O5=11.6  # %P2O5 in purified silvent
PS_SO4=0.07  # %SO4 in purified solvent
WS_P2O5=0.2  # %P2O5 in washed solvent
W_flow=23.4  # Water flow
W_SO4=0.02  # %SO4 in water
Prod_flow=48.4 # =36.7*1.32 Product acid flow
Prod_P2O5=37.6  # %P2O5 in product acid
plant_data = [PS_P2O5, PS_SO4, WS_P2O5, W_flow, W_SO4, Prod_flow, Prod_P2O5]

In [24]:
last=LLC.Stage(30, 15, [12,11.6,0.07],[0,0,0.02], EQUIL_rotem_washing3, eff=0.8)
last.roundoutput()

Oout= 26.6
yout= [12.0, 2.919, 0.029]  %water_org, %solubles
yout_tag= [12.0, 0.748, 0.019]  %water_org_equilib, %solubles_equilib
Aout= 18.37
xout= [1.35, 14.714, 0.089]  %solvent_aq, %solubles
equilibrium efficiency= [1.  0.8 0.8]
error= [7e-07, 2.47e-05, 1.3e-05]
convergence= [-1e-06, -8e-06, -4.4e-06]


In [25]:
opt_bat=Optimal_Washing_Battery(plant_data, stages_num=5)

Calculating entrainment = f(slope,eff) parameters
. . . . . . 

Calculating optimal eff and entrain
. . . . . 
 optimal data:
 eff: 0.9166666666666666 entrain: 3.6564918311335357 convergence: 0.00034266219878038054
Calculating entrainment optimal battery simulation
Ready


In [26]:
opt_bat.summary_table

,Plant data,Model data
PS_flow,159.196491,159.196491
PS_P2O5,11.6,11.6
WS_flow,134.196491,129.695273
WS_P2O5,0.2,0.185617
W_flow,23.4,23.4
Prod_flow,48.4,48.174087
Prod_P2O5,37.6,37.587116
P2O5_recovery,98.5%,98.1%
Stages_num,6,5
Efficiency,---,0.916667


In [27]:
opt_bat.profile_table

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
"Org in, ton/hr",159.196491,146.580907,138.513402,132.558916,130.929254
"y1 in, %",12.000000,12.000000,12.000000,12.000000,12.000000
"y2 in, %",16.011268,9.923873,5.487615,1.855615,0.756006
"y3 in, %",0.070000,0.091900,0.060448,0.026195,0.019127
"Aq in, ton/hr",40.919266,32.540781,26.371553,24.678920,23.400000
"x1 in, %",1.272669,1.273122,1.373511,1.669332,0.000000
"x2 in, %",41.081489,28.113967,12.484515,4.265437,0.000000
"x3 in, %",0.308664,0.247360,0.105692,0.045488,0.020000
"Org out, ton/hr",146.581915,138.505908,132.559451,130.928258,129.695273
"y1 out, %",12.000000,12.000000,12.000000,12.000000,12.000000


In [28]:
opt_bat.profile_table

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
"Org in, ton/hr",159.196491,146.580907,138.513402,132.558916,130.929254
"y1 in, %",12.000000,12.000000,12.000000,12.000000,12.000000
"y2 in, %",16.011268,9.923873,5.487615,1.855615,0.756006
"y3 in, %",0.070000,0.091900,0.060448,0.026195,0.019127
"Aq in, ton/hr",40.919266,32.540781,26.371553,24.678920,23.400000
"x1 in, %",1.272669,1.273122,1.373511,1.669332,0.000000
"x2 in, %",41.081489,28.113967,12.484515,4.265437,0.000000
"x3 in, %",0.308664,0.247360,0.105692,0.045488,0.020000
"Org out, ton/hr",146.581915,138.505908,132.559451,130.928258,129.695273
"y1 out, %",12.000000,12.000000,12.000000,12.000000,12.000000


In [29]:
# opt_bat.profile_table.to_csv('Profile_table.csv')

## Operation Curve

In [30]:
# @markdown Profile data
data=CopyPasteToArray('''
x	y
0	0.0531
3.01	0.26
10.8	1.22
20.69	6.02
33.12	7.77
34.9	8.86
35.81	10.3
''')
profile_data1=pd.DataFrame(data[1:], columns=data[0])
# profile_data1.to_csv('Profile_data.csv')

In [31]:
## P2O5
bat=opt_bat.bat
summary_table=opt_bat.summary_table
fig10_1 = go.Figure()
n=1 #H3PO4
fig10_1.add_trace(go.Scatter(x=[0,summary_table['Plant data'].loc['Prod_P2O5']], y=[summary_table['Plant data'].loc['WS_P2O5'],summary_table['Plant data'].loc['PS_P2O5']], name='Plant data', marker_color='orange'))
fig10_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))
fig10_1.add_trace(go.Scatter(x=profile_data1.x, y=profile_data1.y, name='Plant profile data', marker_color='grey', visible='legendonly'))

fig10_1.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig10_1.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
# fig10_1.update_layout(title='', width=1200, height=600,)
fig10_1.show()

## Equilibrium data

In [32]:
data=CopyPasteToArray('''
H3PO4_aq	SO4_aq	H3PO4_org	water_org	solvent_aq	SO4_org	water_org-original
0.011	0.0932	0.072	8.69	1.70	0.0186	12.09
10.325	0.0017	0.229	10.66	1.52	0.02	10.66
10.380	0.0990	0.269	8.31	1.50	0.0208	10.41
19.821	0.0031	0.635	11.3	1.34	0.01884	11.3
19.710	0.0928	0.773	10.36	1.32	0.0195	12.34
28.710	0.0057	1.435	11.96	1.28	0.01978	11.96
27.882	0.0858	1.325	9.7	1.34	0.0222	11.1
35.611	0.0071	2.871	11.94	1.28	0.036	11.94
35.887	0.0846	2.981	12.02	1.24	0.0374	12.6
43.065	0.0082	5.328	12.12	1.28	0.057	12.12
43.065	0.0802	5.604	10.38	1.06	0.0608	11.7
49.966	0.0074	8.723	12.28	1.28	0.0872	12.28
0.017	0.1540	0.042	11.48	1.46	0.0164	13.28
4.831	0.0020	0.146	12.7	1.78	0.01302	12.6
4.941	0.1078	0.173	12.06	1.66	0.0139	11.84
19.600	0.0033	0.828	11.84	1.38	0.01688	11.42
19.766	0.1120	0.856	12.46	1.38	0.01832	11.2
''')
# data
# data2=pd.DataFrame(data, columns=['H3PO4_aq',	'SO4_aq',	'H3PO4_org',	'water_org',	'solvent_aq',	'SO4_org'])
data3=pd.DataFrame(data[1:], columns=data[0])
# data3_names=CopyPasteToArray('''
# wp12	wp21	wp22	wp31	wp32	wp41	wp42	wp51	wp52	wp61	wp62	wp71	wp13	ww11	ww12	ww21	ww22
# ''')

In [33]:
# data3.to_csv('Lab_data.csv')

In [34]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh4 = x_mesh.flatten().reshape(31*31,1)
y_mesh4 = y_mesh.flatten().reshape(31*31,1)
mesh4=np.concatenate((x_mesh4, y_mesh4), axis=1)

In [35]:
a=np.concatenate((np.zeros([len(mesh4),1]),mesh4), axis=1)

In [36]:
mesh_predict_x=np.array([EQUIL_rotem_washing3([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3([row]).yout for row in a])
mesh_predics4_1=mesh_predict_y[:,:,1]
mesh_predics4_2=mesh_predict_y[:,:,0]
mesh_predics4_3=mesh_predict_x[:,:,0]
mesh_predics4_4=mesh_predict_y[:,:,2]

In [37]:
fig_equil10_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=opt_bat.bat
# H3PO4_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil10_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil10_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil10_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil10_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil10_1.update_scenes(camera_projection_type="orthographic")

fig_equil10_1.show()